# Generate Price Change Data
In this section, daily price data about listings is reshaped in order to convey more information about how price changes of a given listing impact property performance.  Price changes for each listing are extracted and for each price change, the change in average daily revenue resulting from the price change is calculated.

## Import packages and load in data

In [6]:
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.model_selection import train_test_split
import datetime
import seaborn as sns
pd.options.display.max_columns = 100
pd.options.display.max_rows = 1000

In [2]:
# import data
dir = str(Path().resolve())
cal_df = pd.read_csv(dir + "/../data/cal_df.csv")
cal_df['date'] = df['date'].apply(lambda x: datetime.date(int(x[0:4]), int(x[5:7]), int(x[8:10])))

## Define functions to be used to extract average daily revenues

In [123]:
# find the difference between each sequential value in the given list
def get_diff(arr):
    if len(arr) > 0:
        diff = [arr.pop(0)]
        for index in range(len(arr)):
            diff.append(arr[index] - arr[index - 1])
        return diff
    else:
        return arr

In [94]:
# Array[float] -> Array[float]
# given an array of prices, finds the average daily revenue (adr) changes between every price change
def get_adr(prices):
    adr = []
    pc = []
    ind = []
    cur_p = prices[0]
    start_index = 0
    revenue = 0
    for index, p in enumerate(prices):
        if not np.isnan(p):
            if p == cur_p:
                revenue += p
            else:
                adr.append(revenue / (index - start_index))
                pc.append(p - cur_p)
                ind.append(index)
                cur_p = p
                start_index = index
                revenue = 0
    return get_diff(adr), pc, ind

## Implement functions to find average daily revenue changes per price change per listing

In [124]:
# find adr changes for each listing
ids = cal_df.listing_id.unique()
all_dfs = []
for i in ids:
    print(i)
    l = cal_df.loc[cal_df.listing_id==i,:].sort_values(by='date', ascending=True)
    adr, pc, ind = get_adr(l.price.values)
    dates = l.date.iloc[ind]
    id = [l.listing_id.values[0]] * len(adr)
    all_dfs.append(pd.DataFrame(data={'id': id, 'date' : dates, 'revenue_change': adr, 'price_change': pc}))

1810172
6976
3075044
4283698
4085362
225834
7252607
1936861
225979
2583074
6933545
5434353
4025194
1591827
753446
3755609
5884429
879929
1494726
7651065
3420384
8473851
1668313
3575220
4922204
1724900
849408
5759676
3988018
6242681
5498472
7858329
321328
7086825
2843445
6603827
6742139
6513924
8339473
962610
3229255
894539
1861070
3968797
6144896
6400432
4987859
5706985
6069883
8548176
5521378
7929693
6911272
4054079
2219399
3775876
7823025
4586681
6364042
4483549
285270
8354938
1301500
7723492
8083905
1136972
6142209
4405982
5904596
2365632
1701741
5166870
6862594
4331214
5293067
6404106
1692573
5092993
5182313
1110769
4452800
6390613
8336364
381187
6914622
6106955
4149974
6386351
732932
2698996
749247
8212160
6730758
4181530
1552406
8392652
1994683
5101552
5049582
4838985
6425318
830153
2378421
332225
4000384
3021446
8481291
764414
568234
4025268
3905508
7845607
6990904
1067184
973840
4065322
4765070
3079305
7287822
2875267
1071235
2592416
3616081
4105704
2308630
594693
1203014
66022

8253407
8605937
4714840
6017574
414419
7672949
8533464
7698631
1636735
7731797
5454513
6618919
8188894
7740436
8584276
8577620
48612
7585431
5504813
6001339
5992501
7961477
7834063
2441095
1048397
5307640
1497879
1178162
6142396
4315439
4315428
2201031
602975
8212817
8065282
2634361
8253083
5969914
5616179
8326439
8629460
1141522
6261496
4614037
182049
7178893
4868653
2828803
6146559
3615760
5892953
5990808
1042420
8154052
4314054
8252058
5967637
7482338
6344696
8469735
4107959
8624898
7033732
7634067
7035556
5364137
8587249
5478135
1458081
118905
1059604
7678756
1051482
238846
7966317
8410340
3667348
1178106
6337106
4128326
2019292
7680519
4080000
57800
1066767
5940866
6521607
6314998
8268591
1147871
8239805
9273
5744742
8340599
7880828
3917027
7948411
3802530
854542
4820071
8514932
6614984
8589560
6436334
8134231
7984949
5806428
5933463
4638356
7053121
1227322
1587540
3820516
3749523
2167993
5174821
6236230
29765
2798599
619142
8564330
5684598
1658983
5903011
6000531
1445064
4402259


4958693
6031151
7915228
4196643
2022303
5856633
363127
6543678
5999291
4585452
3479660
3792147
3432035
4287348
4738267
8190228
4863101
7927790
451872
5927267
4586590
6151079
3725631
3749277
5026613
4607307
3585343
2021274
802688
1340649
4947526
1887507
4996375
8362075
6705513
8052617
5380160
7826523
5384923
6911945
7330494
2135075
4250871
7988755
6317722
6629066
6296656
4261781
4261784
8274008
4016050
8055839
690109
6310903
5842934
585148
6088708
7611271
5163162
2843596
5511924
4151979
623731
6489209
5358742
395975
8409307
6317446
7841193
1156549
7855120
7933149
994138
4928894
5019306
5054601
646087
6544642
7713728
5103430
7052847
5824015
8076561
8310653
7052171
72811
6823017
3930872
7074932
3897995
8607856
6289262
5870922
7931331
2930663
7460789
95453
341207
2569369
8377490
5355128
6476613
7319961
8238078
8202409
5762866
7868209
8319646
5967594
7245413
5967421
7527677
5589714
7178959
1839549
4278873
2411064
6179374
7479725
8112652
10758
5889190
4745692
1525494
6277566
4786562
7277653


9663423
11841986
8827268
11710680
11686744
9821291
9611852
14821911
9728602
11873789
12496332
5352623
11902134
14461998
10227043
14709217
11397511
13378987
13477286
13378936
11041684
13378923
11771810
6348631
9840919
8480507
9430774
12426977
12508801
10083878
10427882
639111
14933461
8381472
6038520
1106555
14304682
14091605
14819274
7377034
13460331
14770892
14589341
12049787
12034317
13377097
11590512
10656576
9706279
14817983
13605430
10656206
14486599
7095320
12308161
9508975
9197535
12916551
10391455
9574727
14522631
4461846
824244
11060140
14125957
12587887
12653367
8753913
9949819
9853959
10671192
9446242
9474436
9757412
7901248
14094279
8815257
14372667
12979965
13078164
14405817
14589471
13713445
13078068
4787233
12094488
12290886
10734159
5867761
5834930
11470818
13928400
11634638
10656899
13912472
7342489
9487079
14489201
9094794
986399
639101
9207914
11927580
11585459
8782991
13697966
8970393
9864083
11687122
13697464
11796879
12318093
14680277
12812461
10567830
10634364
12

12709469
13800294
13031523
8983081
14864858
8753559
13111691
12833084
11035352
13684319
2526773
14770708
14322546
13192046
11717217
10226171
14912536
9685102
11569156
14698794
9896713
10835494
6865619
13222465
4489752
13156872
13374823
14500012
2294730
8962914
13960293
14402688
11441717
14767573
14303090
14878512
12708781
11662212
14692682
13938959
8265182
601763
13266176
10185866
13028899
13491626
9926796
9301075
13261681
13015767
10279083
9607296
6706463
14685903
14159799
8720673
1757417
10964552
13487348
14552489
12618291
14677678
9340410
9292118
4180178
11626869
13827734
9550398
10127270
13907143
6401859
14593751
3974503
14719754
13296735
11983286
9607385
13275334
13697676
14778561
14272646
890705
10381729
9826706
14086516
8761060
6006328
8904546
12492474
8904297
14686119
14242888
14663819
14664852
9047646
12904318
10234895
3522237
14691657
12971329
5729203
13565183
737519
14092792
7389719
13928082
6763587
14928000
11410652
11376403
3788075
14682145
14747561
14826101
8899828
107642

...

23981561
20997108
22200077
21350397
23745626
23744498
23305456
23903888
23737707
23753345
23716020
23749841
23070890
23752336
23745811
21679585
22435979
22627055
23304072
22030644
24300918
22627141
19223966
7031119
21697785
16238114
21237147
24302130
9361705
23744547
22745898
23743618
22730888
21574183
24300650
22402260
19893604
23304141
21796325
21498863
5822827
4759349
17940615
23741630
21527161
16106584
22714438
23394062
22584746
21312401
22585323
23287047
21682805
22164858
22242630
23752515
21678571
21711401
22228885
23872062
22322123
23787959
22955155
22846729
23918171
22788158
24347269
6200741
22228550
21693386
15288004
24218721
18956699
22987717
22083423
22031411
24347078
21596226
23883271
21448761
23743908
21090146
22718029
22144960
22032913
22175736
22150316
21498966
8831050
21498969
22652823
24201414
22617711
21567155
23304442
23520944
21498860
24398088
22583276
22408657
9970895
20395035
21214295
23629701
23677672
21643899
15870509
23748545
21498971
21361200
21498861
20226352

23797507
21378036
21058704
22229847
22407761
22357836
24248639
22692343
16065865
12265127
21410674
21484100
19872750
23838806
23839637
23838433
20601153
21241180
14081071
21623737
21536088
22154367
21445237
22729849
21624391
23761335
21547645
22084682
13309117
4191592
22639181
21612082
22811301
23396164
21663547
21264930
23892862
24370874
21546411
22369465
21454212
21648941
24154346
21838660
24128226
22231567
23805314
24313446
24138157
23687506
16755558
23999502
23579975
24016169
20441774
22896721
21210513
22981723
23465746
22814137
23544584
23311167
22944278
22981769
23054149
6473970
21952800
22859515
23027286
23027875
15005881
22485102
23149016
21683291
21477647
16583728
21924790
21202826
21953265
23293175
2807040
15723073
21484706
21381927
23627923
21839232
22389994
23633083
23470168
23669430
22345235
22571244
24352165
21729205
23805779
23668146
23747775
21765587
22251665
22810966
24152390
20731975
23576958
21740829
22354274
22828268
21583773
21826608
23948205
21896268
23019378
2315

21604166
24749244
24817963
24477391
24598613
24649475
24963651
3371742
24825394
24464154
24692083
22164240
24888080
24221823
24808907
24499346
10988226
24608314
24649966
24370285
21964456
24244491
24966344
24912611
24985370
25010337
158745
15509837
24489960
23022301
20118368
22903205
24965806
25079379
24243652
24649250
25013220
18624885
22105624
24300286
18829493
24855865
24896294
24809947
24939830
24801839
24944930
24798981
23327519
24855960
24855703
24770791
19296973
24799020
4525875
24161442
24628253
13333573
5559098
25085955
1544144
24914428
24746941
24484501
24992422
24626252
24626250
24623340
24563107
24540681
24161279
24626249
24865433
24961014
24623314
24540981
24770058
24623322
24623318
24736163
24158645
24696205
24885249
24844369
24853642
19589175
21426396
25066653
24827131
24605199
24592772
24377640
24518213
24944280
24945296
24770340
24956556
24945336
24856678
24945374
24846668
25047997
24909410
24541110
24935629
25077703
24008757
24983583
21906430
24945415
24518149
2494051

10524872
10575430
11871190
12211874
12336281
12898177
13057265
13166405
13173362
13588854
13650313
13657486
13972298
14253538
14485918
14927372
15799850
16158151
16159110
16211849
16275852
17006890
17042710
17104147
17585989
17634084
17666410
17669073
17690222
17849739
17907229
18035633
18370244
18501631
18516303
18668081
18785113
19291138
19650444
19801241
19817729
19946681
20230196
20235491
20694378
20996280
21006936
21096288
21141934
21350925
21444743
21471048
21701532
21779767
21781068
21789235
21799387
21800554
21815325
22087346
22525315
22530658
22531126
22533216
22584985
22588076
22612107
22629502
22920165
23022188
23127262
23128151
23166408
23171291
23234070
23375776
23380501
23380964
23483066
23500537
23642565
23739621
23842606
23947734
23965606
23987184
24032465
24034961
24041561
24063318
24117769
24232126
24324950
24369910
24406709
24453629
24476477
24519734
24560253
24632540
24913891
24916335
24916375
24938922
24965808
25235632
25293765
25308568
25335040
25390399
25405673
2

28911895
28637727
28912741
28638612
28639363
28989756
28915006
28915403
28917735
28918898
28992318
28639832
28925755
28932949
28640081
28932963
28640732
28936851
29001707
28640771
28939808
29002249
28640787
28641409
29002914
29003230
28643134
29003469
29003641
29003753
28941687
29003823
28658262
28658394
29004900
29005578
29007744
28954103
28958230
28659843
28663014
29009284
29009425
29009521
28963337
28964046
29009549
29010062
28663016
28964603
28965215
28663740
28664575
28965363
28965672
28967211
28967400
29011576
28664576
29011930
28670717
29014832
28671062
29027894
28968832
29029942
28972885
28687266
29030194
29030261
28691218
29030959
29030965
6600980
29035646
29035657
29035765
28973787
28979819
29037205
28980698
28981396
28571952
29038749
29046191
28984581
27395928
28569308
28583967
29049958
29050084
28572319
28761702
29053226
28572389
29053227
15500671
29054806
29057092
29058232
29059620
28763217
28571342
15481725
22842217
28939517
29060136
28990121
29179789
29062086
28736461
19

## Engineer and export resulting dataset

In [148]:
# group all dataframes from adr change results
df = pd.concat(all_dfs, axis=0, ignore_index=True, sort=True)

In [149]:
# engineer dataframe before writing
df = df.dropna()
df['year'] = df.date.apply(lambda x: x.year)
year_start = df.date.apply(lambda x: datetime.date(int(x.year), 1, 1))
df['days_since'] = (df.date - year_start).apply(lambda x: int(x.days))
df = df.drop('date', axis=1)

In [150]:
df.to_csv(dir + "/../data/price_changes.csv", index=False)